### Word Group In Youtube Firebase

This notebook is used to send language level data to Firebase.

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# sample num
sending_sample_num = 400  # 150,200,300,500

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Spanish
Lang Id = 4
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
##cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
#app = firebase_admin.initialize_app(cred)

In [7]:
#fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(sending_sample_num)
df_28

,video_id,start_time,end_time,text
0,UVeOYNyXd0k,5745,5749,eso de lo que es la la el el
1,rP0PowPzbJE,23,25,si lo que se está es una un
2,l1fO8CJN7Bw,2158,2160,es que por qué no para en
3,2DvDa1DL9iM,97,100,la una que es para los
4,vNrtkGZ9T98,579,581,pero es que no es para
...,...,...,...,...
214,RT68Ueo4L24,132,134,para qué
215,9vgOD0qrdnA,39,40,para qué
216,Rzc3hw1khnQ,195,196,para eso
217,X2zZHJLzo8I,209,211,pero por


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(sending_sample_num)
df_200

,video_id,start_time,end_time,text
0,ZyghXM2t1CM,685,692,porque lo tienes todo pero a la vez es como qu...
1,ujmlPjCimbM,1795,1801,aquí yo no lo voy a hacer porque lo voy a hace...
2,MeAGI1vCdN4,155,161,mucho más o menos así más pero que están muy b...
3,XEsfzSBTXMI,2583,2592,que yo tengo entonces por eso no se puede o se...
4,rlVCOWvwzpw,582,589,así así que bueno si estás aquí hace tiempo ya...
...,...,...,...,...
209,BaN87C1pLlk,548,552,es bueno porque si eres como yo tienes un
210,aJRgQZtIxYQ,194,198,esto lo otro y lo otro de todo un poco te
211,ZpZUcyDxUxk,533,538,pero es lo mejor porque ahí está todo muy
212,ZemxOoEqhtg,186,190,porque gracias a ti estamos donde estamos


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(sending_sample_num)
df_1000

,video_id,start_time,end_time,text
0,QGlH1rRdROo,250,256,también dice que se siente solo porque no tien...
1,O0rkwLyf3BA,1122,1129,así que creo que todavía hay algunas preguntas...
2,fl70SaNinZs,744,751,el problema es que tienen la mejor arma del mu...
3,NWsd2DM3akQ,436,443,si tienes razón pero en realidad te lo dice po...
4,DlUnYlm_NvA,101,107,que además lo único que quería era información...
...,...,...,...,...
204,QJBbHXJPyvk,278,283,con la comida no me quiero sentir mal tampoco ...
205,0TnHJEbN6Hg,41,49,sino es como empezar el viaje con tu amor esa ...
206,ZgqzZQa6gQM,3606,3613,bueno aquí está la parte porque está como en o...
207,EMMIZWcQSEw,1644,1652,estamos bien tenemos vino suficiente con nuest...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(sending_sample_num)
df_5000

,video_id,start_time,end_time,text
0,wFEtvZmaNwQ,207,219,y además cómo sabría el tipo cuándo ha cumplid...
1,fcxTPwMyaNs,490,501,por lo tanto desea comenzar a hacer cosas con ...
2,zTxdXfa8zlA,196,206,y como la tiene en su poder se la vende a max ...
3,Jw5JFE_7icI,425,433,desde luego no es mala idea porque él no está ...
4,NzYHZFCpNag,479,490,la casa es donde está el corazón espero que us...
...,...,...,...,...
199,INSqDuzs3eM,2453,2461,eso no puede seguir pasando tus bienes más me ...
200,mT_NPaMsJfQ,1137,1144,como tú puedes ver estoy vivo eso es porque us...
201,u64WPiPzmfo,108,117,para ustedes sino para cada mismo para mí mism...
202,HYlTJhI9Xng,791,799,le llevamos tres puntos al segundo aunque nos ...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(sending_sample_num)
df_10000

,video_id,start_time,end_time,text
0,xqr2skItS3E,116,127,justamente debajo y para más casualidad termin...
1,zTxdXfa8zlA,461,471,pero sí es cierto que si hubiese cables por en...
2,3dc1Dwjlwys,84,93,qué pasa el chaval jamás hasta ese momento ha ...
3,Jw5JFE_7icI,425,433,desde luego no es mala idea porque él no está ...
4,NzYHZFCpNag,479,490,la casa es donde está el corazón espero que us...
...,...,...,...,...
206,AlgbfFcPWsg,211,218,dientes aquí por donde van esos cinco dientes ...
207,I4tf0f4vmRI,462,468,cuántos tubos necesitaré para tocar una canció...
208,mT_NPaMsJfQ,714,721,dicen vosotros todos parados ahora pero aquí m...
209,K_JWIUtTyO8,259,267,muy interesante pensar como cualquier realidad...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(sending_sample_num)
df_20000

,video_id,start_time,end_time,text
0,Ur8vnsSGa7U,398,410,pero es que si ese era tu plan para qué dejas ...
1,ZCGlkmnwFaE,70,82,todos estos cortes se pueden hacer usando una ...
2,oLuBvogL88U,218,232,incluso después de la muerte el cuerpo humano ...
3,Xq64fmkeETo,296,307,pero todavía podría usarla si cree que su pilo...
4,O0rkwLyf3BA,745,756,las cosas han avanzado en términos de tecnolog...
...,...,...,...,...
208,WoRZzcEpZnw,412,419,porque ya no está incluida dentro del precio n...
209,ulIMb24WRyY,327,338,para mejorar algunos controles sobre bancos pe...
210,qFkOQTtt7yY,465,472,si queréis os enseñan unos ejercicios iguales ...
211,NDfBwFFQvC8,124,130,nuestra mente se vuelve menos eficiente pero c...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(sending_sample_num)
df_40000

,video_id,start_time,end_time,text
0,haz50gsL2G4,438,452,o por qué no fabricar una bomba propia si tien...
1,Ur8vnsSGa7U,398,410,pero es que si ese era tu plan para qué dejas ...
2,haz50gsL2G4,385,396,no hay ninguna seguridad más porque conseguir ...
3,danPB1e3MhA,548,561,dependiendo del tipo de clip seleccionado en l...
4,30pBgk4cZU8,449,462,en la parte incorrecta del motor lo que causa ...
...,...,...,...,...
239,1mSvyIZyLXA,269,277,américa licencia vemos acá colombia también es...
240,A21lPWo9YNw,36,44,colombiana pero me estoy conectando desde pana...
241,QxKOJ0ywq4o,276,284,una estrategia similar aunque tomemos infinita...
242,n1RvthD0f04,401,405,puede aportar o si tiene alguna prueba gratuit...


##### Calculate Speech Time Mean

In [16]:
(df_28["end_time"] - df_28["start_time"]).mean()

2.493150684931507

In [17]:
(df_200["end_time"] - df_200["start_time"]).mean()

5.093457943925234

In [18]:
(df_1000["end_time"] - df_1000["start_time"]).mean()

6.937799043062201

In [19]:
(df_5000["end_time"] - df_5000["start_time"]).mean()

7.416666666666667

In [20]:
(df_10000["end_time"] - df_10000["start_time"]).mean()

7.815165876777251

In [21]:
(df_20000["end_time"] - df_20000["start_time"]).mean()

8.103286384976526

In [22]:
(df_40000["end_time"] - df_40000["start_time"]).mean()

8.55327868852459

#### Sending Data To Firestore

In [23]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [24]:
fire_db = firestore.client()

##### Language Data

In [25]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [26]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [27]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [28]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [29]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [30]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [31]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [32]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [33]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [34]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [35]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [36]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [37]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [38]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [39]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [40]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [41]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [42]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [43]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [44]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [45]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [46]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [47]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [48]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [49]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [50]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [51]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [52]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [53]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [54]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [55]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [56]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [57]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1685372338
  nanos: 808819000
}